# PIC-SURE API use-case: Phenome-Wide analysis on COPDgene data

This is a tutorial notebook, aimed to be quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

Databases exposed through PIC-SURE API encompass a wide heterogeneity of architectures and data organizations underneath. PIC-SURE hide this complexity and expose the different databases in the same format, allowing researchers to focus on the analysis and medical insights, thus easing the process of reproducible sciences.

### More about PIC-SURE
PIC-SURE stands for Patient-centered Information Commons: Standardized Unification of Research Elements. The API is available in two different programming languages, python and R, allowing investigators to query databases in the same way using any of those languages.

PIC-SURE is a large project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface, allowing research scientist to get quick knowledge about variables and data available for a specific data source.

The python API is actively developed by the Avillach-Lab at Harvard Medical School.

GitHub repo:
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client



 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the get_your_token.ipynb notebook. It contains explanation about how to get a security token, mandatory to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later (although earlier versions of Python 3 must work too)
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### IPython magic command

Those two lines of code below do load the `autoreload` IPython extension. Although not necessary to execute the rest of the Notebook, it does enable to reload every dependency each time python code is executed, thus enabling to take into account changes in external file imported into this Notebook (e.g. user defined function stored in separate file), without having to manually reload libraries. Turns out very handy when developing interactively. More about [IPython Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [1]:
%load_ext autoreload
%autoreload 2

### Installation of required python packages

Using the pip package manager, we install the packages listed in the `requirements.txt` file.

In [2]:
!cat requirements.txt

numpy>=1.17.3
matplotlib>=3.1.1
pandas>=0.25.3
scipy>=1.3.1
tqdm>=4.38.0
statsmodels>=0.10.2
git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
git+https://github.com/hms-dbmi/pic-sure-python-client.git 


In [3]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/pip-req-build-e4qzo858
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git /private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/pip-req-build-e4qzo858
  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/pip-req-build-lj7rk_2j
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-client.git /private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/pip-req-build-lj7rk_2j
  Created wheel for PicSureHpdsLib: filename=PicSureHpdsLib-0.9.0-py2.py3-none-any.whl size=17755 sha256=d869400dac0911765501ab2c720fe64c6cd73650c97517993a9c07ba5f4f0a7d
  Stored in directory: /private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/pip-ephem-wheel-cache-f5o1wtu1/wheels/6c/ac/12/4d142709d83e0bdd96c661e183f2dcbb0e03f90e98e5ac256e
  Created wh

Import all the external dependencies, as well as user-defined functions stored in the `python_lib` folder

In [4]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, match_dummies_to_varNames, joining_variablesDict_onCol
from python_lib.HPDS_connection_manager import tokenManager

In [5]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureClient: 0.1.0\n- PicSureHpdsLib: 1.1.0\n")
print("The PIC-SURE API libraries versions you've been downloading are: \n- PicSureClient: {0}\n- PicSureHpdsLib: {1}".format(PicSureClient.__version__, PicSureHpdsLib.__version__))

NB: This Jupyter Notebook has been written using PIC-SURE API following versions:
- PicSureClient: 0.1.0
- PicSureHpdsLib: 1.1.0

The PIC-SURE API libraries versions you've been downloading are: 
- PicSureClient: 0.1.0
- PicSureHpdsLib: 1.1.0


##### Set up the options for displaying tables and plots in this Notebook

In [6]:
# Pandas DataFrame display options
pd.set_option("max.rows", 435)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)

### Connecting to a PIC-SURE network

Several information are needed to get access to data through the PIC-SURE API: a network URL, a resource id, and a user security token which is specific to a given URL + resource.

In [7]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [8]:
with open(token_file, "r") as f:
    my_token = f.read()

In [9]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects are created here: a `connection` and a `resource` object, using respectively the `picsure` and `hpds` libraries. 

As we will only be using one single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter** (FYI, the `connection` object is useful to get access to different databases stored in different resources). 

It is connected to the specific data source ID we specified, and enables to query and retrieve data from this source.

### Getting help with the PIC-SURE python API

Each object exposed by the PicSureHpdsLib library got a `help()` method. Calling it will print out a helper message about it. 

In [10]:
resource.help()


        [HELP] PicSureHpdsLib.useResource(resource_uuid)
            .dictionary()       Used to access data dictionary of the resource
            .query()            Used to query against data in the resource
            
        [ENVIRONMENT]
              Endpoint URL: https://biodatacatalyst.integration.hms.harvard.edu/picsure/
             Resource UUID: 02e23f52-f354-4e8b-992c-d37c8b9ba140


For instance, this output tells us that this `resource` object got 2 methods, and it gives insights about their function. 

## Using the *variables dictionnary*

Once connection to the desired resource has been established, we first need to get a quick grasp of which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance offers the possibility to retrieve matching records according to a specific term, or to retrieve information about all available variables, using the `find()` method. For instance, looking for variables containing the term `COPD` is done this way: 

In [11]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("COPD")

Subsequently, objects created by the `dictionary.find` exposes the search result using 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [25]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:5],
        "Entries": dictionary_search.entries()[0:5]})

{'Count': 425,
 'Entries': [{'HpdsDataType': 'phenotypes',
              'categorical': False,
              'max': 87.8387,
              'min': 0.0214,
              'name': '\\Genetic Epidemiology of COPD (COPDGene)\\Subject '
                      'Phenotype\\CT Acquisition Parameters\\CT '
                      'Slicer\\Percent gas trapping total lung: CT Slicer\\',
              'observationCount': 8276,
              'patientCount': 8276},
             {'HpdsDataType': 'phenotypes',
              'categorical': True,
              'categoryValues': ['Do not know', 'No', 'Yes'],
              'name': '\\Genetic Epidemiology of COPD (COPDGene)\\Subject '
                      'Phenotype\\Respiratory Disease\\Family History\\Asthma: '
                      'Father or Mother\\Father: asthma\\',
              'observationCount': 10098,
              'patientCount': 10098},
             {'HpdsDataType': 'phenotypes',
              'categorical': True,
              'categoryValues': [

In [13]:
dictionary_search.DataFrame().head()

,min,categorical,observationCount,patientCount,max,HpdsDataType,categoryValues
KEY,,,,,,,
\Genetic Epidemiology of COPD (COPDGene)\Subject Phenotype\CT Acquisition Parameters\CT Slicer\Percent gas trapping total lung: CT Slicer\,0.0214,False,8276,8276,87.8387,phenotypes,None
\Genetic Epidemiology of COPD (COPDGene)\Subject Phenotype\Respiratory Disease\Family History\Asthma: Father or Mother\Father: asthma\,NaN,True,10098,10098,NaN,phenotypes,"[Do not know, No, Yes]"
\Genetic Epidemiology of COPD (COPDGene)\Subject Phenotype\Respiratory Disease\Respiratory Conditions\Hayfever\Hayfever: diagnosed by doctor or other health professional\,NaN,True,3000,3000,NaN,phenotypes,"[Do not know, No, Yes]"
\Genetic Epidemiology of COPD (COPDGene)\Subject Phenotype\Respiratory Disease\Educational and Occupational History\Work/ Education\Highest degree or level of school completed\,NaN,True,10098,10098,NaN,phenotypes,"[8th grade or less, College or technical schoo..."
\Genetic Epidemiology of COPD (COPDGene)\Subject Phenotype\Respiratory Disease\Respiratory Conditions\Hayfever\Hayfever: have you ever had hay fever (allergy involving nose or eyes)\,NaN,True,10099,10099,NaN,phenotypes,"[Do not know, No, Yes]"


**`.DataFrame()` enables to get the result of the dictionary search in a pandas DataFrame format. This way, it enables to:** 


* Use the various criteria exposed in the dictionary (patientCount, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variables names, to be used in the query, as shown below.

Variable names, as currently implemented in the API, aren't handy to use right away.
1. Very long
2. Presence of backslashes that requires modification right after copy-pasting. 

However, using the dictionary to select variables can help to deal with this. Hence, one way to proceed is to retrieve the whole dictionary in the form of a pandas DataFrame, as below:

In [14]:
plain_variablesDict = resource.dictionary().find().DataFrame()

Indeed, using the `dictionary.find()` function without arguments return every entries, as shown in the help documentation.

In [15]:
resource.dictionary().help()


        [HELP] PicSureHpdsLib.Client(connection).useResource(uuid).dictionary()
            .find()                 Lists all data dictionary entries
            .find(search_string)    Lists matching data dictionary entries
        


In [16]:
plain_variablesDict.iloc[10:20,:]

,min,categorical,observationCount,patientCount,max,HpdsDataType,categoryValues
KEY,,,,,,,
\Cardiovascular Health Study (CHS) Cohort\YR7\Psychological and Psychiatric Observations\Cognitive Status\3MSE\TO REMEMBER 'SHOES' 3MSE Score Component\,NaN,True,10,5,NaN,phenotypes,"[AFTER 'WAS IT SHOES, SHIRT, OR SOCKS', SPONTA..."
"\Atherosclerosis Risk in Communities (ARIC) Cohort\Subject Phenotype\Cohort Exam\Personal History Form. PHX\Visit 4\Physical Observations\Medical Care\[Medical care]. Has a doctor ever said you had any of the following? Cancer. Q6f [Personal History form, exam 4]\",NaN,True,12,12,NaN,phenotypes,"[No, Yes]"
\Multi-Ethnic Study of Atherosclerosis (MESA) Cohort\MESA Air\MESA Air New Recruit Intima-Media Thickness (IMT)\RIGHT POSTERIOR OBLIQUE LUMEN STD DEV\,6.080465e-02,False,9,9,4.262806e-01,phenotypes,None
"\Multi-Ethnic Study of Atherosclerosis (MESA) Cohort\MESA Classic\MESA Exam 1 Main\Health and Life\FELT EVERYTHING WAS AN EFFORT, PAST WEEK\",1.000000e+00,False,5799,5799,4.000000e+00,phenotypes,None
\Cardiovascular Health Study (CHS) Cohort\BASE2\Physical Measurements and Observations\ECG\LV WALL MOTION- SEMIQUANTITATIVE\,NaN,True,4,4,NaN,phenotypes,[NORMAL]
"\Framingham Cohort\Lab Work\Urine Chemistries\Urinary Biomarkers, Offspring Cohort Exam 6. The following biomarkers were measured: alpha-1- and beta-2-microglobulins, calbindin, clusterin, connective tissue growth factor, cystatin-c, glutathione s-transferase alpha, kidney injury molecule-1, neutrophil gelatinase-associated lipocalin, osteopontin, trefoil factor 3, Tamm-Horsfall urinary glycoprotein, tissue inhibitor of metalloproteinases 1, and vascular endothelial growth factor.\Measurable range indicator for VEGF\",NaN,True,2303,2303,NaN,phenotypes,"[below measurable range, within measurable range]"
"\Atherosclerosis Risk in Communities (ARIC) Cohort\Subject Phenotype\Cohort Exam\Stroke\Stroke\Exam 4\Sociodemography and Administration\ARIC participant ID [Stroke, exam 4]\",1.310000e+08,False,10864,10864,1.312000e+08,phenotypes,None
\The Jackson Heart Study (JHS)\Subject Phenotype\mribV3\Sociodemography and Administration\31. Mean Wall Thickness Descending Arch in (mm)\,1.800000e+00,False,617,617,5.100000e+00,phenotypes,None
\Cardiovascular Health Study (CHS) Cohort\CHS.v4 and Later - Phenotype Variables\SHHS 1 - PSG\Physical Measurements and Observations\Lung\Apnea\Apnea - Obstructive\Maximum Obstructive Apnea duration in REM (Back position) 2% desaturation\,0.000000e+00,False,1144,1144,2.160000e+02,phenotypes,None


The dictionary currently returned by the API provide various information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if categorical, False if continuous/numerical
- min/max: only provided for non-categorical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

#### Variable dictionary + pandas multiIndex

Though helpful, we can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information and ease dealing with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrate how to quickly scan an select groups of related variables.

Printing the 'multiIndexed' variable Dictionary allows to quickly see the tree-like organisation of the variables. Moreover, original and simplified variable names are now stored respectively in the "varName" and "simplified_varName" columns.

In [17]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [18]:
variablesDict

simplified_varName  \
level_0                                           level_1                                            level_2                                            level_3 level_4 level_5 level_6 level_7 level_8 level_9 level_10 level_11 level_12                                                      
Atherosclerosis Risk in Communities (ARIC) Cohort Samples                                            Sample ID                                          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                               Sample ID   
                                                                                                     Sample Use                                         NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                              Sample Use   
                                                                                                     Sample source                                      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                           Sample source   
                                                                                                     Source sample ID                                   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                        Source sample ID   
                                                                                                     Study ID                                           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                                Study ID   
...                                                                                                                                                                                                                                                                                       ...   
Women's Health Initiative                         Subject Sample Mapping, Sample Aliases, Sample ... Sample use. Array_DNA_Methylation: Genome-wide ... NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN       Sample use. Array_DNA_Methylation: Genome-wide...   
                                                                                                     Source repository where samples originate (Cori... NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN       Source repository where samples originate (Cor...   
                                                                                                     WHI Sample ID                                      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                           WHI Sample ID   
                                                                                                     WHI Subject ID                                     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                                          WHI Subject ID   
_Consents                                         Short Study Accession with Consent Code            NaN                                                NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN      NaN                 Short Study Accession with Consent Code   

                                                                                                                                                                                                                                                                                      varName  \
level_0                                           level_1                                            level_2                                            level_3 level_4 level_5 level_6 level_7 level_8 level_9 level_10 lev

In [19]:
# Now that we have seen how our entire dictionnary looked, we limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in every variables pertaining to the "Medical history" and "Medication history" subcategories.

In [20]:
mask_medication = variablesDict.index.get_level_values(2) == "Medication History"
mask_medical = variablesDict.index.get_level_values(2) == "Medical History"
medication_history_variables = variablesDict.loc[mask_medical | mask_medication,:]
medication_history_variables

simplified_varName  \
level_0                                  level_1           level_2         level_3                   level_4                     level_5                      level_6 level_7                                            level_8 level_9 level_10 level_11 level_12                                                      
Cardiovascular Health Study (CHS) Cohort BASE2             Medical History Cardio-Cerebro-Vascular   Decrease - Loss of Function BLACKOUTS OR FAINTING        NaN     NaN                                                NaN     NaN     NaN      NaN      NaN                                   BLACKOUTS OR FAINTING   
                                                                                                                                 DATE OF MOST RECENT EPISODE  NaN     NaN                                                NaN     NaN     NaN      NaN      NaN                             DATE OF MOST RECENT EPISODE   
                                                                                                                                 DIABETIC                     NaN     NaN                                                NaN     NaN     NaN      NaN      NaN                                                DIABETIC   
                                                                                                                                 DURATION OF LONGEST EPISODE  NaN     NaN                                                NaN     NaN     NaN      NaN      NaN                             DURATION OF LONGEST EPISODE   
                                                                                                                                 HEADACHE (varname: SCHHDC22) NaN     NaN                                                NaN     NaN     NaN      NaN      NaN                            HEADACHE (varname: SCHHDC22)   
...                                                                                                                                                                                                                                                                                                                ...   
The Jackson Heart Study (JHS)            Subject Phenotype Medical History Medical History Form, MHX Visit 9                     F. Diagnostic Procedures     Q49     Q49b1. F. Diagnostic procedures. What was the r... NaN     NaN     NaN      NaN      NaN       Q49b1. F. Diagnostic procedures. What was the ...   
                                                                                                                                                                      Q49c. F. Diagnostic procedures. Since last Jack... NaN     NaN     NaN      NaN      NaN       Q49c. F. Diagnostic procedures. Since last Jac...   
                                                                                                                                                                      Q49c1. F. Diagnostic procedures. What was the r... NaN     NaN     NaN      NaN      NaN       Q49c1. F. Diagnostic procedures. What was the ...   
                                                                                                                                                                      Q49d. F. Diagnostic procedures. Since last Jack... NaN     NaN     NaN      NaN      NaN       Q49d. F. Diagnostic procedures. Since last Jac...   
                                                                                                                                                                      Q49d1. F. Diagnostic procedures. What was the r... NaN     NaN     NaN      NaN      NaN       Q49d1. F. Diagnostic procedures. What was the ...   

                                                                                                                                                                                                                 

Although pretty simple, it can be easily combined with other filters to quickly select necessary variables.

## Querying and retrieving data

Beside from the dictionary, the second cornerstone of the API is the `query` object. It is the entering point to retrieve data from the resource.

In [21]:
my_query = resource.query()

The query object got several methods that enable to build a query

- The `query.select().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, without any record (ie subjects/rows) subsetting.

- The `query.require().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all the variables passed, and only records that do not contain any null values for those variables.

- The `query.anyof().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, and only records that do contain at least one non-null value for those variables.

- The `query.filter().add()` method accept variable names a variable name as strings as argument, plus additional values to filter on that given variable. The query will return this variable and only the records that do match this filter.

All those 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

#### Building the query

In [22]:
mask = variablesDict["simplified_varName"] == "How old were you when you completely stopped smoking? [Years old]"
yo_stop_smoking_varname = variablesDict.loc[mask, "varName"] 

In [23]:
mask_cat = variablesDict["categorical"] == True
mask_count = variablesDict["observationCount"].between(100,2000)
varnames = variablesDict.loc[mask_cat & mask_count, "varName"]

In [24]:
my_query.filter().add(yo_stop_smoking_varname, min=20, max=70)
my_query.select().add(varnames)

KeyboardInterrupt: 

## Retrieving the data

Once our query object is finally built, we use the `query.run` function to retrieve the data corresponding to our query

In [ ]:
query_result = my_query.getResultsDataFrame()

In [ ]:
query_result.shape

In [ ]:
query_result.head()